### Tutorial on Space-time FEM with FEniCSx
Dominik Kern ORCID [0000-0002-1958-2982](https://orcid.org/0000-0002-1958-2982) 

This notebook is a supplement to the tutorial with doi [10.5281/zenodo.16761463](https://zenodo.org/records/16761463)

**solving the non-dimensional wave equation in a 1D bar using space-time finite-elements with the formulation as second-order equation** 

In [ ]:
import numpy as np
import ufl
from dolfinx import geometry
from dolfinx import fem, mesh, plot
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI
from petsc4py.PETSc import ScalarType
import pyvista as pv

#### parameters

In [ ]:
nx = 4
nt = 8
order = 1  # Polynomial order

#### discretization

In [ ]:
domain = mesh.create_unit_square(MPI.COMM_WORLD, nx, nt, mesh.CellType.quadrilateral)
V = fem.functionspace(domain, ("Lagrange", order))

# Locate facets for boundary conditions (initial time and left/right boundaries)
fdim = domain.topology.dim - 1
facets_t0 = mesh.locate_entities_boundary(domain, fdim, lambda x: np.isclose(x[1], 0.0))
facets_space = mesh.locate_entities_boundary(domain, fdim, lambda x: np.logical_or(np.isclose(x[0], 0.0), np.isclose(x[0], 1.0)))

# Initial displacement u(x, 0) = u_0(x)
def u0_func(x):
    return np.sin(np.pi * x[0])
u_initial = fem.Function(V)
u_initial.interpolate(u0_func)
dofs_t0 = fem.locate_dofs_topological(V, fdim, facets_t0)
bc_t0 = fem.dirichletbc(u_initial, dofs_t0)

# Initial velocity v(x, 0) = v_0(x) = 0
v0 = fem.Function(V)
v0.interpolate(lambda x: np.zeros_like(x[0]))

# Spatial boundary u(0,t) = u(1,t) = 0
dofs_space = fem.locate_dofs_topological(V, fdim, facets_space)
bc_space = fem.dirichletbc(ScalarType(0.0), dofs_space, V)

bcs = [bc_t0, bc_space]

u = ufl.TrialFunction(V)
Du = ufl.TestFunction(V)

# Source term f
f = fem.Constant(domain, ScalarType(0.0))

# Define the boundary integral for the initial velocity, mark the t=0 boundary with tag 1
T0_MARKER = 1
facet_tags = mesh.meshtags(domain, fdim, facets_t0, np.full_like(facets_t0, T0_MARKER))
ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tags)

# Bilinear form
a = (u.dx(0) * Du.dx(0) - u.dx(1) * Du.dx(1)) * ufl.dx
L = f * Du * ufl.dx + v0 * Du * ds(T0_MARKER)

##### solution

In [ ]:
problem = LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

#### post-processing

In [ ]:
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(V)
u_grid = pv.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
plotter = pv.Plotter()
u_warped = u_grid.warp_by_scalar()
plotter.add_mesh(u_warped, show_edges=True, scalar_bar_args={"vertical":True})
if not pv.OFF_SCREEN:
    plotter.show_grid(xlabel="x", ylabel="t", zlabel="u")
    plotter.show()